<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here
# Display data types, non-null counts, and missing values for each column
summary = df.info()

# Check the number of missing values for each column
missing_values = df.isnull().sum()

# Display the missing values count
print("Missing values per column:")
print(missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
Missing values per column:
ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10631
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here
# Generate basic statistics for numerical columns
numerical_summary = df.describe()

# Display the summary statistics
print(numerical_summary)



         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000         0.00000   
25%          0.000000 

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here
# Check unique values in the 'Country' column
unique_countries = df['Country'].unique()

# Display the unique countries
print("Unique countries:")
print(unique_countries)

# Check for any other potential issues like unexpected entries
country_counts = df['Country'].value_counts()
print("\nCountry value counts:")
print(country_counts)


Unique countries:
['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 'Norway'
 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria' 'Switzerland'
 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia' 'Brazil'
 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France' 'Saudi Arabia'
 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden' 'Netherlands' 'Croatia'
 'Pakistan' 'Czech Republic' 'Republic of North Macedonia' 'Finland'
 'Slovakia' 'Russian Federation' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'United Republic of Tanzania' 'Hungary' 'Argentina' 'Portugal'
 'Sri Lanka' 'Latvia' 'China' 'Singapore' 'Lebanon' 'Spain' 'South Africa'
 'Lithuania' 'Viet Nam' 'Dominican Republic' 'Indonesia' 'Kosovo'
 'Morocco' 'Taiwan' 'Georgia' 'San Marino' 'Tunisia' 'Bangladesh'
 'Nigeria' 'Liechtenstein' 'Denmark' 'Ecuador' 'Malaysia' 'Albania'
 'Azerbaijan' 'Chile' 'Ghana' 'Peru' 'Bolivia' 'Egypt' 'Luxembourg'
 'Montenegro' 'Cyprus' 'Paraguay' 'Kazakhstan' 'Sl

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [7]:
## Write your code here
# Create a dictionary to map inconsistent country names to standardized values
country_mapping = {
    'USA': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'UK': 'United Kingdom',
    'United States of America': 'United States',
    'U.S.': 'United States',
    'India, US': 'United States',
    # Add more mappings as needed
}

# Standardize the 'Country' column using the mapping
df['Country'] = df['Country'].replace(country_mapping)

# Check if standardization worked
print(df['Country'].unique())


['United States' 'United Kingdom' 'Canada' 'Norway' 'Uzbekistan' 'Serbia'
 'Poland' 'Philippines' 'Bulgaria' 'Switzerland' 'India' 'Germany'
 'Ireland' 'Italy' 'Ukraine' 'Australia' 'Brazil' 'Japan' 'Austria'
 'Iran, Islamic Republic of...' 'France' 'Saudi Arabia' 'Romania' 'Turkey'
 'Nepal' 'Algeria' 'Sweden' 'Netherlands' 'Croatia' 'Pakistan'
 'Czech Republic' 'Republic of North Macedonia' 'Finland' 'Slovakia'
 'Russian Federation' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'United Republic of Tanzania' 'Hungary' 'Argentina' 'Portugal'
 'Sri Lanka' 'Latvia' 'China' 'Singapore' 'Lebanon' 'Spain' 'South Africa'
 'Lithuania' 'Viet Nam' 'Dominican Republic' 'Indonesia' 'Kosovo'
 'Morocco' 'Taiwan' 'Georgia' 'San Marino' 'Tunisia' 'Bangladesh'
 'Nigeria' 'Liechtenstein' 'Denmark' 'Ecuador' 'Malaysia' 'Albania'
 'Azerbaijan' 'Chile' 'Ghana' 'Peru' 'Bolivia' 'Egypt' 'Luxembourg'
 'Montenegro' 'Cyprus' 'Paraguay' 'Kazakhstan' 'Slovenia' 'Jordan'
 'Venezuela, Bolivarian Republic of...' 'Costa Rica

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [8]:
## Write your code here
# Apply one-hot encoding to the 'Employment' column
df = pd.get_dummies(df, columns=['Employment'], drop_first=True)

# Check the first few rows to verify the encoding
print(df.head())


   ResponseId                      MainBranch                 Age RemoteWork  \
0           1  I am a developer by profession  Under 18 years old     Remote   
1           2  I am a developer by profession     35-44 years old     Remote   
2           3  I am a developer by profession     45-54 years old     Remote   
3           4           I am learning to code     18-24 years old        NaN   
4           5  I am a developer by profession     18-24 years old        NaN   

    Check                                   CodingActivities  \
0  Apples                                              Hobby   
1  Apples  Hobby;Contribute to open-source projects;Other...   
2  Apples  Hobby;Contribute to open-source projects;Other...   
3  Apples                                                NaN   
4  Apples                                                NaN   

                                             EdLevel  \
0                          Primary/elementary school   
1       Bachelor’s deg

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [9]:
## Write your code here
# Identify columns with the highest number of missing values
missing_values = df.isnull().sum()

# Sort columns by the number of missing values (descending order)
missing_values_sorted = missing_values.sort_values(ascending=False)

# Display the columns with the highest number of missing values
print("Columns with the highest number of missing values:")
print(missing_values_sorted)


Columns with the highest number of missing values:
AINextMuch less integrated                                              64289
AINextLess integrated                                                   63082
AINextNo change                                                         52939
AINextMuch more integrated                                              51999
EmbeddedAdmired                                                         48704
                                                                        ...  
Employment_Student, full-time;Student, part-time;Employed, part-time        0
Employment_Student, full-time;Student, part-time;Retired                    0
Employment_Student, part-time                                               0
Employment_Student, part-time;Employed, part-time                           0
Employment_Student, part-time;Retired                                       0
Length: 222, dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [10]:
## Write your code here
# Impute missing values in the 'ConvertedCompYearly' column using the median
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(df['ConvertedCompYearly'].median())

# Print the first few rows to verify the result
print(df[['ConvertedCompYearly']].head())


   ConvertedCompYearly
0              65000.0
1              65000.0
2              65000.0
3              65000.0
4              65000.0


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [11]:
## Write your code here
# Impute missing values in the 'RemoteWork' column with the most frequent value
df['RemoteWork'] = df['RemoteWork'].fillna(df['RemoteWork'].mode()[0])

# Print the first few rows to verify the result
print(df[['RemoteWork']].head())


                             RemoteWork
0                                Remote
1                                Remote
2                                Remote
3  Hybrid (some remote, some in-person)
4  Hybrid (some remote, some in-person)


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [13]:
## Write your code here
!pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to 'ConvertedCompYearly' column
df['ConvertedCompYearly_normalized'] = scaler.fit_transform(df[['ConvertedCompYearly']])

# Print the first few rows to check the result
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_normalized']].head())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 177.4 MB/s eta 0:00:0000:01
   ConvertedCompYearly  ConvertedCompYearly_normalized
0              65000.0                        0.003998
1              65000.0                        0.003998
2              65000.0                        0.003998
3              65000.0                        0.003998
4              65000.0                        0.003998


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [14]:
## Write your code here
import numpy as np

# Apply log transformation to 'ConvertedCompYearly' column (avoid log(0) by adding 1 to values)
df['ConvertedCompYearly_log'] = np.log1p(df['ConvertedCompYearly'])

# Print the first few rows to check the result
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_log']].head())


   ConvertedCompYearly  ConvertedCompYearly_log
0              65000.0                11.082158
1              65000.0                11.082158
2              65000.0                11.082158
3              65000.0                11.082158
4              65000.0                11.082158


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [ ]:

# Create a new column 'ExperienceLevel' based on 'YearsCodePro' column
def assign_experience_level(years):
    if years <= 2:
        return 'Beginner'
    elif years <= 5:
        return 'Intermediate'
    elif years <= 10:
        return 'Experienced'
    else:
        return 'Expert'

# Apply the function to the 'YearsCodePro' column
df['ExperienceLevel'] = df['YearsCodePro'].apply(assign_experience_level)

# Print the first few rows to check the result
print(df[['YearsCodePro', 'ExperienceLevel']].head())


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
